In [ ]:
from numpy import * 
import scipy as sp 
from matplotlib import pyplot as plt
from numpy import random as RN
from scipy.linalg import expm, norm
from scipy.special import erfinv
from scipy.stats import linregress 
from scipy.integrate import odeint, simps
from scipy.optimize import curve_fit
import time 
from alive_progress import alive_bar as abar 
plt.rc('text', usetex=True)
plt.rc('font', family='serif') 

In [ ]:
# %% List of Global System Parameters 
global vs, vxyzt, isp, N, lenT, Time, dt, ks, rs, kmeant, rmeant, tpskip 
global e, j, m0, hbar, ep0, g, a, c, kB, sx, sy, sz, Eg, Eso, EgL, Gamma
global ep, m, T, kT, muB, A, b, E, E0, eta, ye, ae, epInf, Bx, Kx, aKf, bKf, cKf
global Ni, Da, aNP, dens, vSnd, f, fk, vsat, ksat, w0, EtaR
global Ndis, Ed, Nc, Gammaflag, fdis, factor, selNdis, selT, selNi

# %% Data Collection Samples
Ts = array([100, 150, 200, 250, 300, 350, 400, 500])
Nis = array([3.5e22, 1e22, 1e23, 1e24, 1e25])
Ndiss = array([5e12, 1e9, 1e10, 1e11, 1e12, 1e13])
selT = 4
selNi = 0
selNdis = 1 

In [ ]:
# %% Defining Global System Parameters
#------------------------------ Simulation Params --------------------------------#
N = int(500)        # Number of injected Carriers 
lenT = 2000          # Number of Time Points
T = Ts[selT]             # Temperature in K
Ni = Nis[selNi]           # Impurity density (n-type) 
Ndis = Ndiss[selNdis]       # Dislocation density (negatively charged)
Gamma = 5e14     # Auto-Calibrable Scattering rate
fdis = 1.0       # Probablity of filling up a dislocation site with an electron
isp = 1.0        # Initial x-Spin polarization 
tpskip = 10     # How much time points to be skipped  
factor = 0.05 

#---------------------------- Universal Consts -----------------------------------#

e = sp.constants.elementary_charge  # Electronic Charge
m0 = sp.constants.electron_mass     # Electron Mass in Free Space
kB = sp.constants.Boltzmann  # Boltzmann Const
hbar = sp.constants.hbar # Modified Plank's Const
ep0 = sp.constants.epsilon_0  # Permittivity of the Free Space
g = 2                          # Gyromagnetic Ratio for Electron
sx = matrix([ [0, 1], [1, 0 ] ]) 
sy = matrix([ [0, -1j], [1j, 0] ])  
sz = matrix([[ 1, 0],[ 0, -1 ]])       # Pauli Matrices     
kT = kB*T

#--------------------------------- GaN Params ------------------------------------#

a = 3.186e-10                   # Fundamental Lattice Const
c = 5.186e-10                   # Basal Lattice Const
Eg = ( 3.427 - ( ((9.39e-4)*T*T) / (T + 772) ) )*e   # Bandgap 
EgL = 4.9*e                     # B.G at L minima 
Eso = 0.008*e                   # Conduction band split
ep = 8.9*ep0                    # DC Dielectric Const
epInf = 5.35*ep0                # HF Dieletric Const
m = 0.22*m0                      # Effective mass (0.22m0)
muB = e*hbar/(2*m)              # Bohr Magneton 
aNP = (e/Eg)*(1 - m/m0)**2      # Non-parabolicity Const
eta = Eso/(Eso+Eg)                
# Dresselhaus SOC term for G minima 
ye = lambda k: e*1e-30*(0.32 - 0.10038874766*exp(0.8922080847*norm(array([k[0], k[1]]))/1e10)) 
ae = 0.009*e*1e-10              # 9 meV.A        (Rashba SOC term for G minima)
Bx = array([3.959, 4.02, 4.08, 4.2, 4.326])     # 5 points from B vs k graph 
Kx = array([0, 0.122, 0.164, 0.24, 0.3])        # Kx's unit is per A
def ParaFitFunc(x,A,B,C):
    return A + B*x + C*x*x
abc, covmat = curve_fit(ParaFitFunc, Kx, Bx)
aKf, bKf, cKf = abc                            # Parabolic Fitting of Bx vs Kx 
Da = 9.2*e                      # Deformation potential
dens = 6.15e3                   # Material denasity (kg/m3) of GaN
vSnd =6.59e3                    # Sound velocity through GaN
w0 = 1.383689667e14             # Optical Phonon Energy hbar*w0 = 91.2 meV
# Boltzmann Dist Const
A = sqrt(m/(2*pi*kT))
b = m/(2*kT) 
# Bias Field
E0 = 1e5   # 1 kV/cm
E = array([E0, 0, 0])     #  E-field
f = e*E/m                 #  Acceleration
fk = e*E/hbar 
vsat = e*E/(m*Gamma)
ksat = e*E/(hbar*Gamma)
Ed = (12e-3)*e   # Donor activation energy
Nc = 2.3e24      # Effective DOS in Cond band 
EtaR = - E*(e*hbar/m)*pi*Eso*(Eso + 2*Eg)/(Eg*(Eg + Eso)*(3*Eg + 2*Eso)) 

In [ ]:
# %% Necessary Functions
#---------------------------- Random Number Generator -------------------------------#
def R():
    return RN.rand(1)[0]

#-------------------------------- Material Functions ---------------------------------#
def B(k):
    global aKf, bKf, cKf
    kt = norm(array([k[0], k[1]]))*1e-10 
    return aKf + bKf*kt + cKf*kt*kt

# Screening parameter (1/Screening length)  
def lmbda():
    global e, Ni, ep, kT, Ndis, Nc, kT, c, fdis, Ed
    phid = Nc*exp(-Ed/(2*kT))
    nn = -0.5*(phid + fdis*Ndis/c) + sqrt( ( 0.5*(phid + fdis*Ndis/c) )**2 + phid*(Ni - fdis*Ndis/c) )
    np = nn*(2-nn/Ni)
    return sqrt((e*e*np)/(ep*kT))

# Position of dislocation energy level above valence band 
def trapLevel():
    global fdis, ep, c, Eg, kT, e, Nc, Ed, Ni, Ndis 
    phid = Nc*exp(-Ed/(2*kT))
    nn = -0.5*(phid + fdis*Ndis/c) + sqrt( ( 0.5*(phid + fdis*Ndis/c) )**2 + phid*(Ni - fdis*Ndis/c) )
    Ef = Eg - (kT/e)*log(Nc/nn)
    Eo = e*e/(4*pi*ep*c)
    Ex = 3*fdis*log(fdis/(c*c*c*sqrt(pi*Ni)))
    ET = 0.232*fdis - Ef - Eo*Ex
    return ET*e 

def gamma(k):
    global aNP, e, m, hbar
    knorm = norm(k) 
    Gam = ((hbar*knorm)**2)/(2*m) 
    return e*(-1 + sqrt(1 + 4*Gam*aNP/e))/(2*aNP)


In [ ]:
# ---------------- PDF Functions -------------------- #

def POP_Pdf(th, k):
    global aNP, e, m, hbar, w0
    knorm = norm(k)
    Ek0 = hbar*w0/e
    Gam = ((hbar*knorm)**2)/(2*m); 
    Ek = (-1 + sqrt(1 + 4*Gam*aNP/e))/(2*aNP); 
    knum = ((sqrt(1 + 2*((aNP/e)*(hbar*knorm)**2)/m) + sign(Ek0-Ek)*2*(aNP/e)*hbar*w0)**2) -1
    kden = 2*(aNP/e)*hbar*hbar/m
    kmod = sqrt(knum/kden)
    Gamp = ((hbar*kmod)**2)/(2*m)
    Ekp = (-1 + sqrt(1 + 4*Gamp*aNP/e)) / (2*aNP) 
    b = (knorm*knorm + kmod*kmod)/(2*knorm*kmod)
    ak = sqrt( (1 + aNP*Ek) / (1 + 2*aNP*Ek) )
    akp = sqrt( (1 + aNP*Ekp) / (1 + 2*aNP*Ekp) )
    ck = sqrt( (aNP*Ek) / (1 + 2*aNP*Ek) )
    ckp = sqrt( (aNP*Ekp) / (1 + 2*aNP*Ekp) )
    a = (ak*akp)/(ck*ckp)
    ths = linspace(0, 2*pi, 500)
    P = lambda th: ( ((a+b)**2)*log((b - cos(th))/(b - 1)) - 4*(sin(th/2)**2)*(a + 0.5*b + 0.5*(cos(th/2)**2)) )                                             
    Pdf = lambda th: P(th)/simps(P(ths), ths)
    return Pdf(th) 


def Dis_Pdf(phi, k):
    b = 2*(norm(k)/lmbda())**2
    a = ((2*b +1)**1.5)/(2*pi*(b + 1))
    phis = linspace(0, 2*pi, 200)
    P = lambda phi: a/(((2*b*(sin(phi))**2) + 1)**2)
    Pdf = lambda phi: P(phi)/simps(P(phis), phis)
    return Pdf(phi) 


def WBoltz(k):
    global aNP, m, hbar, kT, N 
    E = gamma(k) 
    return exp(-E/kT)


In [ ]:
# ----------- Plotting PDFs ------------ #
# POP vs theta
ths = linspace(0, 2*pi, 1000)
kk1 = array([1, 0, 0])*5e7         
kk2 = array([1, 0, 0])*1.2e8      
kk3 = array([1, 0, 0])*2e8               
Ekk1, Ekk2, Ekk3 = gamma(kk1), gamma(kk2), gamma(kk3)  
plt.figure(1, dpi=300)  
plt.plot(ths*180/pi, POP_Pdf(ths, kk1), 'b', label = r'$E_k$ = ' + str(int(10000*1000*Ekk1/e)/10000) + ' meV')  
plt.plot(ths*180/pi, POP_Pdf(ths, kk2), 'g', label = r'$E_k$ = ' + str(int(1000*1000*Ekk2/e)/1000) + ' meV')   
plt.plot(ths*180/pi, POP_Pdf(ths, kk3), 'r', label = r'$E_k$ = ' + str(int(1000*1000*Ekk3/e)/1000) + ' meV')  
plt.legend() 
plt.xlabel(r'Polar Angle, $\theta$ (degrees)') 
plt.ylabel(r'$P_\mathrm{pop}(\theta)$') 


# Dis_PDF vs Phi 
phi = linspace(0, 2*pi, 1000) 
kk1 = array([1, 0, 0])*5e7         
kk2 = array([1, 0, 0])*1.2e8      
kk3 = array([1, 0, 0])*2e8     
Ekk1, Ekk2, Ekk3 = gamma(kk1), gamma(kk2), gamma(kk3)  
plt.figure(2, dpi=300) 
plt.semilogy(phi*180/pi, Dis_Pdf(phi, kk1), 'b', label = r'$E_k$ = ' + str(int(10000*1000*Ekk1/e)/10000) + ' meV')  
plt.semilogy(phi*180/pi, Dis_Pdf(phi, kk2), 'g', label = r'$E_k$ = ' + str(int(1000*1000*Ekk2/e)/1000) + ' meV')   
plt.semilogy(phi*180/pi, Dis_Pdf(phi, kk3), 'r', label = r'$E_k$ = ' + str(int(1000*1000*Ekk3/e)/1000) + ' meV')  
plt.legend() 
plt.xlabel(r'Azimuthal Angle, $\phi$ (degrees)') 
plt.ylabel(r'$P_\mathrm{dis}(\phi)$')  
plt.show() 

In [ ]:
#---------------------------- Scattering Functions W(k) ------------------------------#

def AcousticW(k):
    global m, kT, Da, dens, vSnd, hbar, aNP, e
    Gam = ((hbar*norm(k))**2)/(2*m)  
    E = gamma(k) 
    Fa = ( (1+aNP*E/e)**2 + (1/3)*(aNP*E/e)**2 )/( (1+ 2*aNP*E/e)**2 )
    W = ((((2*m)**1.5)*kT*Da*Da)/(2*pi*dens*vSnd*vSnd*((hbar)**4)))*sqrt(Gam)*(1+ 2*aNP*E/e)*Fa
    return W


def ImpurityW(k):
    global Ni, e, ep, kT, hbar, m, aNP, N 
    Gam = ((hbar*norm(k))**2)/(2*m)
    E = gamma(k) 
    Nk = (m**1.5)*sqrt(Gam)*(1+ 2*aNP*E/e)/(sqrt(2)*pi*pi*(hbar**3))
    lmb = lmbda()
    F = ((1*e*e/ep)**2)*Nk*Ni/(32*hbar*(norm(k)**4))
    c = (lmb/(2*norm(k)))**2
    W = 4*pi*F*(1/c)/(1+c)
    return W


def PolOptPhnW(k):
    global e, m, w0, hbar, epInf, ep, aNP, kT 
    Gam = ((hbar*norm(k))**2)/(2*m)
    E = gamma(k) 
    Nw0 = 1/( exp(hbar*w0/kT) -1 )
    # Absorption (when E <= hbar*w0) else Emission
    if E <= hbar*w0:
        Ep = E + hbar*w0
    else:
        Ep = E - hbar*w0
        Nw0 = Nw0 + 1
    Gamp = Ep*(1+aNP*Ep/e)
    A = ( 2*(1+aNP*E/e)*(1+aNP*Ep/e) + aNP*(Gam+Gamp)/e )**2
    B = (-2*aNP*sqrt(Gam*Gamp)/e)*( 4*(1+aNP*E/e)*(1+aNP*Ep/e) + aNP*(Gam+Gamp)/e )
    C = 4*(1+aNP*E/e)*(1+aNP*Ep/e)*(1+ 2*aNP*E/e)*(1+ 2*aNP*Ep/e)
    F0 = (A*log(abs((sqrt(Gam)+sqrt(Gamp))/(sqrt(Gam)-sqrt(Gamp)))) +B)/C
    W = (e*e*sqrt(m)*w0/(4*pi*sqrt(2)*hbar))*(1/epInf - 1/ep)*(1+ 2*aNP*Ep/e)*F0/sqrt(Gam)
    return W


def DislocW(k):
    global hbar, ep, c, Ndis, m, e, fdis
    Ld = 1/lmbda()
    tauD = (hbar**3)*((ep*c)**2)*( ( 1 + (2*Ld*norm(array([k[0], k[1]])))**2 )**1.5)/(Ndis*m*(fdis**2)*(e*Ld)**4) 
    return 1/tauD 

In [ ]:
# Plotting Scattering Rates, W(k) vs Ek

kk = linspace(1e4, 2e8, 500) 
Ekk = 1000*array([gamma(ki*array([1,1,1])) for ki in kk])/e 
W_acs = array([AcousticW(ki*array([1,1,1])) for ki in kk]) 
W_pop = array([PolOptPhnW(ki*array([1,1,1])) for ki in kk]) 
W_dis = array([DislocW(ki*array([1,1,1])) for ki in kk]) 
W_imp = array([ImpurityW(ki*array([1,1,1])) for ki in kk]) 
skip = 0    
plt.figure(3, dpi=200)
plt.semilogy(Ekk[skip:], W_acs[skip:], label = r'$W_\mathrm{acs}(k)$')   
plt.semilogy(Ekk[skip:], W_dis[skip:], label = r'$W_\mathrm{dis}(k)$') 
plt.semilogy(Ekk[skip:], W_pop[skip:], label = r'$W_\mathrm{pop}(k)$') 
plt.semilogy(Ekk[skip:], W_imp[skip:], label = r'$W_\mathrm{imp}(k)$')    
plt.legend() 
plt.xlabel('Energy (meV)')
plt.ylabel(r'Scattering Rate ($s^{-1}$)')

In [ ]:
#---------------------------------- Scattering Update Functions k -----------------------------#    
def AcousticUpdate(k):
    global hbar, vSnd, aNP, m, e
    knorm = norm(k)
    th = arccos(1-2*R())
    phi = 2*pi*R()
    qnum = 4*hbar*vSnd*((1 + 2*(aNP/e)*((hbar*knorm)**2)/m )**0.5) - 4*hbar*hbar*knorm*cos(th)/m
    qden = 2*hbar*hbar/m - 4*(aNP/e)*((hbar*vSnd)**2)
    q = abs(qnum/qden)
    wq = vSnd*q
    Ekq = hbar*wq/e
    Gam = ((hbar*norm(k))**2)/(2*m)
    Ek = (-1+sqrt(1+ 4*Gam*aNP/e))/(2*aNP)
    kmod = sqrt(knorm**2 + q**2 + sign(Ekq-Ek)*2*knorm*q)
    kp = kmod*array([sin(th)*cos(phi), sin(th)*sin(phi), cos(th)])
    kp = Scat2RealAxis(kp)
    return kp
    
      
def DislocUpdate(k):
    b = 2*(norm(k)/lmbda())**2
    a = ((2*b +1)**1.5)/(2*pi*(b + 1))
    phis = linspace(0, 2*pi, 200)
    P = lambda phi: a/(((2*b*(sin(phi))**2) + 1)**2)
    Pdf = lambda phi: P(phi)/simps(P(phis), phis)
    # Von Neumann Method
    while 1:
        R1 = R()*2*pi
        R2 = R()*max(Pdf(phis))
        if R2 <= Pdf(R1):
            phi = R1
            break
    kr = sqrt(k[0]**2 + k[1]**2)
    kp = array([kr*cos(phi), kr*sin(phi), k[2]])
    return kp    
    
    
def ImpurityUpdate(k):
    kmod = norm(k)
    phi = 2*pi*R()
    a = 1 + 0.5*(lmbda()/kmod)**2 
    Rn = R()
    th = arccos( (1 + a*(1 - 2*Rn))/(a + (1 - 2*Rn)) )
    kp = kmod*array([sin(th)*cos(phi), sin(th)*sin(phi), cos(th)])
    kp = Scat2RealAxis(kp)
    return kp    
    
    
def PolOptPhnUpdate(k):
    global aNP, e, m, hbar, w0
    phi = 2*pi*R()
    knorm = norm(k)
    Ek0 = hbar*w0/e
    Gam = ((hbar*knorm)**2)/(2*m); 
    Ek = (-1 + sqrt(1 + 4*Gam*aNP/e))/(2*aNP); 
    knum = ((sqrt(1 + 2*((aNP/e)*(hbar*knorm)**2)/m) + sign(Ek0-Ek)*2*(aNP/e)*hbar*w0)**2) -1
    kden = 2*(aNP/e)*hbar*hbar/m
    kmod = sqrt(knum/kden)
    Gamp = ((hbar*kmod)**2)/(2*m)
    Ekp = (-1 + sqrt(1 + 4*Gamp*aNP/e)) / (2*aNP) 
    b = (knorm*knorm + kmod*kmod)/(2*knorm*kmod)
    ak = sqrt( (1 + aNP*Ek) / (1 + 2*aNP*Ek) )
    akp = sqrt( (1 + aNP*Ekp) / (1 + 2*aNP*Ekp) )
    ck = sqrt( (aNP*Ek) / (1 + 2*aNP*Ek) )
    ckp = sqrt( (aNP*Ekp) / (1 + 2*aNP*Ekp) )
    a = (ak*akp)/(ck*ckp)
    ths = linspace(0, pi, 200)
    P = lambda th: ( ((a+b)**2)*log((b - cos(th))/(b - 1)) - 4*(sin(th/2)**2)*(a + 0.5*b + 0.5*(cos(th/2)**2)) )                                             
    Pdf = lambda th: P(th)/simps(P(ths), ths)
    # Von Neumann Method
    while 1:
        R1 = R()*pi
        R2 = R()*max(Pdf(ths))
        if R2 <= Pdf(R1):
            th = R1
            break
    kp = kmod*array([cos(phi)*sin(th), sin(phi)*sin(th), cos(th)])
    kp = Scat2RealAxis(kp)
    return kp    


#------------------- Mapping into real axis from scat-axis ------------------------#
    
def Scat2RealAxis(k):
    Cx = sqrt(k[1]**2 + k[2]**2)
    Cy = sqrt( ( k[1]**2 + k[2]**2 )**2 + ( k[0]*k[1] )**2 + ( k[0]*k[2] )**2 ) 
    Cz = norm(k) 
    T11 = 0
    T12 = (k[1]**2 + k[2]**2)/Cy  
    T13 = k[0]/Cz
    T21 = -k[2]/Cx
    T22 = -k[0]*k[1]/Cy        
    T23 = k[1]/Cz
    T31 = k[1]/Cx
    T32 = -k[0]*k[2]/Cy        
    T33 = k[2]/Cz
    T = array([ [T11, T12, T13], [T21, T22, T23], [T31, T32, T33] ])
    return matmul(T,k)


In [ ]:
# --------------------------- Selection of Scattering Mechanisms ------------------------------- #
# Enumeration of Scattering Mechanisms goes like the following.        
# (0) Acoustic Phonon, (1) Polar Optical phonon  , (2) Impurity
# (3) Dislocation, (4) Self Scattering. 
# Random Selection of Scat-Mechanism using Monte Carlo Algo
        
def ScatSelect(k):
    global Gamma, Gammaflag
    W = zeros(5)
    W[0] = AcousticW(k)
    W[1] = PolOptPhnW(k)
    W[2] = ImpurityW(k)
    W[3] = DislocW(k)
    W[4] = Gamma - sum(W[0:4])
    if W[4]<=0:
        print('Recalibrating Gamma at least greater than ', sum(W[0:4]))
        Gamma = 1.1*sum(W[0:4])         # Auto-Calibration of Gamma 
        Gammaflag = 0
    Wc = cumsum(W)
    GamR = R()*Gamma
    iDcheck = where((Wc-GamR)>=0)[0]
    if not sum(iDcheck):
        iD = len(Wc)-1
    else:
        iD = iDcheck[0]
    return iD


# Updating k using the selected mechanisms
def ScatUpdate(k, iD): 
    if (iD==0):
        return AcousticUpdate(k)
    elif (iD==1):
        return PolOptPhnUpdate(k)
    elif (iD==2):
        return ImpurityUpdate(k)
    elif (iD==3):
        return DislocUpdate(k)
    elif (iD==4):
        return k 


# Ensemble Scattering update of k        
def ScatCarr():
    global ks, N
    for i in range(N):
        iD = ScatSelect(ks[i,:])
        ks[i,:] = ScatUpdate(ks[i,:], iD)


In [ ]:
# ------------------------- Mean Functions -------------------------------# 
def MeanK(k, NumComp):
    global N
    if NumComp == 0:
        sumk, wgt = 0, 0
        for i in range(N):
            sumk += k[i,0]*WBoltz(k[i,0])
            wgt += WBoltz(k[i,0])
        return sumk/wgt
    elif NumComp == 1:
        sumk, wgt = 0, 0
        for i in range(N):
            sumk += k[i,1]*WBoltz(k[i,1])
            wgt += WBoltz(k[i,1])
        return sumk/wgt
    elif NumComp == 2:
        sumk, wgt = 0, 0
        for i in range(N):
            sumk += k[i,2]*WBoltz(k[i,2])
            wgt += WBoltz(k[i,2])
        return sumk/wgt
    elif NumComp == 210:
        sumk, wgt = 0, 0
        for i in range(N):
            sumk += norm(k[i,:])*WBoltz(k[i,:])
            wgt += WBoltz(k[i,:])
        return sumk/wgt
    else:
        return mean(k)


def MeanR(r):
    global N
    sumR = sum(r, axis=0)
    return sumR/N 


# -------------------------- Plot Functions ---------------------- #
def ShowDist(k, fign, issave=0):  
    global N, T
    ks_plt = zeros(N);
    for i in range(N):
        ks_plt[i] = norm(k[i,:])  
    plt.figure(fign, dpi=200) 
    plt.hist(ks_plt, bins = 50, density=1)
    plt.xlim(min(ks_plt), max(ks_plt))
    plt.xlabel(r'Momentum, $|k|$ (m$^{-1}$)', fontsize=18)
    plt.ylabel(r'Probability Density', fontsize=18)
    if issave: 
        figname = 'K_Dist_' + str(fign) + str(T) + 'K.png' 
        plt.savefig(figname, dpi=400)


def ExpFitT(x, y, isplot, fign, xLabel, yLabel, tf, issave=0):
    global selT, selNdis, selNi
    ival = y[0]
    def FitFunc(t, yinf, tau):
        return yinf + (ival-yinf)*exp(-t/tau)
    yinf = y[len(y)-1]
    tau = tf              # Initial Guess
    YinfTau, CovMat = curve_fit(FitFunc, x, y, p0 = (yinf, tau))
    yinf, tau = YinfTau
    xx = x
    yy = FitFunc(xx, yinf, tau)
    if isplot: 
        xx = xx/tf
        plt.figure(fign, dpi=200)
        plt.plot(x/tf, y, 'b', xx, yy, 'g')
        plt.xlabel(xLabel, fontsize=14)
        plt.ylabel(yLabel,fontsize=14)
        figname = yLabel + str(selT) + '_Ndis_' + str(selNdis) + '_Ni_' + str(selNi) + '.png'
        if issave:
            plt.savefig(figname, dpi=400)
    RMSE = sqrt(mean(abs(y-yy)))
    return tau, RMSE


def ExpFitR(x, y, isplot, fign, xLabel, yLabel, xf, issave=0):
    global selT, selNdis, selNi
    ival = y[0]
    def FitFunc(t, yinf, tau):
        return yinf + (ival-yinf)*exp(-t/tau)
    yinf = y[len(y)-1]
    tau = xf              # Initial Guess
    YinfTau, CovMat = curve_fit(FitFunc, x, y, p0 = (yinf, tau))
    yinf, tau = YinfTau
    xx = x
    yy = FitFunc(xx, yinf, tau)
    if isplot: 
        xx = xx/xf
        plt.figure(fign, dpi=200) 
        plt.plot(x/xf, y, 'b', xx, yy, 'g')
        plt.xlabel(xLabel, fontsize=14)
        plt.ylabel(yLabel,fontsize=14)
        if issave:
            figname = yLabel + str(selT) + '_Ndis_' + str(selNdis) + '_Ni_' + str(selNi) + '.png' 
            plt.savefig(figname, dpi=400)
    RMSE = sqrt(mean(abs(y-yy)))
    return tau, RMSE


def SinFitT(x, y, isplot, fign, xLabel, yLabel, tf, issave=0):
    global selT, selNdis, selNi 
    def FitFunc(t, A, tau, w, delta):
        return A*exp(-t/tau)*sin(w*t + delta) 
    A = max(abs(y))
    tau = tf              # Initial Guess
    w = 1/tau
    delta = pi/2 
    ATauWDelta, CovMat = curve_fit(FitFunc, x, y, p0 = (A, tau, w, delta))
    A, tau, w, delta = ATauWDelta
    xx = x
    yy = FitFunc(xx, A, tau, w, delta) 
    if isplot: 
        plt.figure(fign, dpi=200) 
        plt.plot(x/tf, y, 'b', xx/tf, yy, 'g')
        plt.xlabel(xLabel, fontsize=14)
        plt.ylabel(yLabel,fontsize=14)
        if issave:
            figname = yLabel+'_T_' + str(selT) + '_Ndis_' + str(selNdis) + '_Ni_' + str(selNi) + '.png'
            plt.savefig(figname, dpi=400)
    RMSE = sqrt(mean(abs(y-yy)))
    return tau, RMSE


In [ ]:
#------------------------------ Time Vector Set-up ----------------------------------#
def TimeSet():
    global lenT, dt, Time, Gamma
    Time = zeros(lenT)
    dt = zeros(lenT-1)
    for i in range(len(dt)):
        dt[i] = -log(1-R())/Gamma;
    Time[1:len(Time)] = cumsum(dt)

# ---------------------------- Carrier Initialization --------------------------------#  
def CarInit(isshow):
    global rs, ks, vs, vxyzt, N, lenT, A, b, m, kT, hbar, isp, kmeant, rmeant, Gammaflag, tpskip 
    Gammaflag = 1
    rs, ks, kmeant, rmeant = zeros([N,3]), zeros([N,3]), zeros(lenT), zeros([lenT,3]) 
    vs = zeros([N, 3])
    vs[:,0] = isp*ones(N)
    vxyzt = zeros([lenT, 3])
    vxyzt[0:tpskip+1,:] = isp*array([1,0,0])
    for i in range(N):
        ux =  sqrt( -2*kT*log( 1-R() ) / m )
        uy = (1/sqrt(b))*erfinv( (2*R()/A)*sqrt(b/pi) - 1 )
        uz = (1/sqrt(b))*erfinv( (2*R()/A)*sqrt(b/pi) - 1 )
        ks[i,:] = array([ux, uy, uz])*m/hbar
    kmeant[0] = MeanK(ks, 210) 
    if isshow==1:
        ShowDist(ks, 1, 0)    

# ------------------------------- Newtonian Update -----------------------------------# 
def UpdateK(t):
    global N, ks, rs, f, fk, hbar, m
    for i in range(N):
        u = hbar*ks[i,:]/m 
        delk = fk*t
        ks[i,:] = ks[i,:] + delk
        delr = u*t + 0.5*f*t**2
        rs[i,:] = rs[i,:] + delr

def UpdateK2(t):
    global N, ks, rs, hbar, m, Gamma, vsat, ksat
    for i in range(N):
        k0 = ks[i,:]
        u = hbar*k0/m
        delr = vsat*t + ((u - vsat)/Gamma)*(1-exp(-Gamma*t))
        rs[i,:] = rs[i,:] + delr
        ks[i,:] = ksat + (k0 - ksat)*exp(-Gamma*t)

In [ ]:
# ----------------------------- Spin Polarization Updater -----------------------------#
# -------------------- Solution of Coupled eq --> dS/dt = OMG X S -------------------- #
def soc_function(ks):
    global BetaL, aNP, hbar, m, eta, Eg, vs, vx, vy, vz, vL, N, lenT, sx, sy, sz, kT, dt, e, factor, g, ae, ye, EtaR
    OMGD = (g*ye(ks)/hbar)*(B(ks)*ks[2]*ks[2] - ks[0]*ks[0] - ks[1]*ks[1])*array([ ks[1]-ks[2], ks[2]-ks[0], ks[0]-ks[1] ]) 
    OMGR = (g*ae/hbar)*array([ ks[1]-ks[2], ks[2]-ks[0], ks[0]-ks[1] ]) 
    OMGE = 2*cross(EtaR, ks)
    return factor*(OMGR + OMGD + OMGE) 

def SpinFinder(tp):
    global ks, BetaL, aNP, hbar, m, eta, Eg, vs, vx, vy, vz, vL, N, lenT, sx, sy, sz, kT, dt, e, factor, g, ae, ye, EtaR
    W, vavg = zeros(N), zeros(3) 
    t = dt[tp]
    for i in range(N):
        OMG = soc_function(ks[i,:])  
        def f(v, t):
            f1 = OMG[2]*v[1] - OMG[1]*v[0]
            f2 = -OMG[2]*v[0] + OMG[0]*v[2]
            f3 = OMG[1]*v[0] - OMG[0]*v[1]
            return [f1, f2, f3]
        t_slv = linspace(0, t, 3) 
        vs[i,:] = odeint(f, vs[i,:], t_slv)[len(t_slv)-1, :]  # Solution of the coupled ODE
        Gam = ((hbar*norm(ks[i,:]))**2)/(2*m)
        En = e*(-1 + sqrt(1 + 4*Gam*aNP/e))/(2*aNP)
        W[i] = exp(-En/kT)
        vavg += vs[i,:]*W[i]
    vavg = vavg/sum(W)                                        # Ensemble Avg by Boltz-Dist 
    vxyzt[tp+1,:] = vavg 

In [ ]:
# %% Flight Scatter Loop
t1 = time.time()  
while 1: 
    tpcount = 0                # tpskip counter 
    TimeSet()                  # Defining Time Grid
    CarInit(1)                 # Initialize k and spin of the injected Carriers 
    with abar(lenT-1, title='Progress') as bar: 
        for tp in range(len(dt)):        
            UpdateK(dt[tp])        # Free flight 
            ScatCarr()             # Random Scattering of Carriers
            if not Gammaflag:      # Chaching if any W(k) exceeds Gamma
                break              # if exceeds, break the loop and re-define Gamma 
            kmeant[tp+1] = MeanK(ks, 0)            # Storing avg k at the runtime
            rmeant[tp+1,:] = MeanR(rs) 
            if tpcount>=tpskip:
                SpinFinder(tp)              # Updates vx, vy, vz, vL at the runtime
            tpcount += 1                    # increment of tpskip counter 
            bar()  
            #print(tp+1, Time[tp+1], abs(vx[tp+1]))
        if Gammaflag:        # If Gamma is OK, then break the inifinity loop
            break            # else restart the simulation with the redefined Gamma 

t2 = time.time() 
print('Time Elapsed', (t2-t1), ' seconds') 

In [ ]:
# %% Retrieve taus 
t = Time[tpskip:len(Time)]
t = t - t[0] 

# Final Momentum Distribution
ShowDist(ks, 1, 0) 

# Spin Relaxation Time
taux, rmsex = ExpFitT(t, vxyzt[tpskip:len(Time), 0], 1, 2, 'Time (ps)', r'$S_x(t)$', 1e-12, 0)    # x-pol
#tauy, rmsey = SinFitT(t, vxyzt[tpskip:len(Time), 1], 1, 3, 'Time (ps)', r'$S_y(t)$', 1e-12, 0)    # y-pol
#tauz, rmsez = SinFitT(t, vxyzt[tpskip:len(Time), 2], 1, 4, 'Time (ps)', r'$S_z(t)$', 1e-12, 0)    # z-pol
tauL, rmseL = ExpFitT(t, norm(vxyzt[tpskip:len(Time), :], axis=1), 1, 5, 'Time (ps)', r'$|S(t)|$', 1e-12, 0) 
taus = tauL                    
print('Spin Ralaxation Time = ',taus*1e12, ' ps') 

# Momentum
tauk, rmsek = ExpFitT(Time[1:30]-Time[1], kmeant[1:30], 1, 6, 'Time (fs)', r'$\mathbf{k}(t)$', 1e-14, 0)   # Momentum Relaxation 
meuk = tauk*e/m
print('Simulation Mobiity = ', meuk*1e4, 'sq-cm/V')

# Spin Diffusion Length
tau_rx, rmse_rx = ExpFitR(array(rmeant)[:,2][tpskip:], vxyzt[tpskip:len(Time), 0], 1, 7, r'Distance (x), $\mu$m', r'$S_x(x)$', 1e-6, 0)
print('Spin Diffusion Length, Ds =', tau_rx)  